# Gathering Data from https://www.altafsir.com/

In [1]:
import pandas as pd
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import io
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

In [38]:
#Install Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2x/qhy93fs134s_883czg81ywrm0000gn/T/ipykernel_81381/2409737876.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## Verse Translation

In [3]:
SURAHS = []

for j in (21,17,10,8,7,5,4,3):
    work = []
    url = 'https://www.altafsir.com/ViewTranslations.asp?Display=yes&SoraNo=1&Ayah=0&toAyah=0&Language=2&LanguageID=2&TranslationBook='+str(j)
    driver.get(url)
    select = Select(driver.find_element('id','TranslationBook'))
    selected_option = select.first_selected_option
    print(selected_option.text)
    work.append(selected_option.text)
    
    for i in range(1, 115):
        url = 'https://www.altafsir.com/ViewTranslations.asp?Display=yes&SoraNo=' + str(i) + '&Ayah=0&toAyah=0&Language=2&LanguageID=2&TranslationBook='+str(j)
        driver.get(url)
        
        driver.switch_to.frame(driver.find_element(By.TAG_NAME, "iframe"))
        work.append(driver.find_element(By.TAG_NAME, "body").text)
        print(i)
    SURAHS.append(work)
    

Muhammad Tahir-ul-Qadri
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
Recitations of Ibrahim walk from saheeh international
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
Royal Aal al-Bayt Institute Translation
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


In [4]:
len(SURAHS)

8

In [5]:
SURAHS[0][1]

'The Opening\nAll praise be to Allah alone, the Sustainer of all the worlds,\n[1:1]\nAll praise be to Allah alone, the Sustainer of all the worlds,\n\n[1:2]\nMost Compassionate, Ever-Merciful,\n\n[1:3]\nMaster of the Day of Judgment.\n\n[1:4]\n(O Allah!) You alone do we worship and to You alone do we look for help.\n\n[1:5]\nShow us the straight path,\n\n[1:6]\nThe path of those upon whom You have bestowed Your favours,\n\n[1:7]\nNot of those who have been afflicted with wrath, nor of those who have gone astray.'

## Cleaning Text

In [6]:
clean_surahs = []
for j in SURAHS:
    work = []
    for i in j:
        surah = i.split("\n")
        ayats = ""
        for i in range(len(surah)):
            if i == 0:
                #print(surah[i])
                ayats += surah[i] + "\n"
            elif i == 1:
                continue
            else:
                if len(surah[i]) > 0:
                    if surah[i][0] == '[':
                        continue
                    else:
                        #print(surah[i])
                        ayats += surah[i] + "\n"
        ayats = ayats[:-1]
        work.append(ayats.split("\n"))
    clean_surahs.append(work)

In [7]:
clean_surahs[0][0]

['Muhammad Tahir-ul-Qadri']

In [8]:
surah_s = []
ayat_s = []
name_s = []
translation = []
worker=[]

for j in clean_surahs:
    
    work = j[0][0]
    surah_num = 0
    for i in j:
        ayat_num = 1
        for k in range(1, len(i)):
            worker.append(work)
            name_s.append(i[0])
            surah_s.append(surah_num)
            ayat_s.append(ayat_num)
            translation.append(i[k])
            #print(worker, name_s, surah_s, ayat_s)
            ayat_num += 1
        surah_num+=1

In [9]:
len(surah_s)

59289

In [12]:
df = pd.DataFrame(data=zip(worker, name_s, surah_s, ayat_s, translation), columns=['Worker', 'Name', 'Surah', 'Ayat', 'Verse'])
df.head()

,Worker,Name,Surah,Ayat,Verse
0,Muhammad Tahir-ul-Qadri,The Opening,1,1,"All praise be to Allah alone, the Sustainer of..."
1,Muhammad Tahir-ul-Qadri,The Opening,1,2,"Most Compassionate, Ever-Merciful,"
2,Muhammad Tahir-ul-Qadri,The Opening,1,3,Master of the Day of Judgment.
3,Muhammad Tahir-ul-Qadri,The Opening,1,4,(O Allah!) You alone do we worship and to You ...
4,Muhammad Tahir-ul-Qadri,The Opening,1,5,"Show us the straight path,"


In [11]:
df.tail()

,Worker,Name,Surah,Ayat,Verse
59284,Marmaduke Pickthall,Mankind,114,2,"The King of mankind,"
59285,Marmaduke Pickthall,Mankind,114,3,"The God of mankind,"
59286,Marmaduke Pickthall,Mankind,114,4,"From the evil of the sneaking whisperer,"
59287,Marmaduke Pickthall,Mankind,114,5,"Who whispereth in the hearts of mankind,"
59288,Marmaduke Pickthall,Mankind,114,6,Of the jinn and of mankind.


In [19]:
df.to_csv("Quran_English.csv", index=False)

# Tafaseer

In [20]:
import pandas as pd

In [21]:
df = pd.read_csv("Quran_English.csv")

In [78]:
#Install Driver
driver = webdriver.Chrome(ChromeDriverManager().install())

/var/folders/2x/qhy93fs134s_883czg81ywrm0000gn/T/ipykernel_81381/2409737876.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [ ]:
TAFSEERS = []
url = 'https://www.altafsir.com/Tafasir.asp?tMadhNo=0&tTafsirNo=74&tSoraNo=1&tAyahNo=1&tDisplay=yes&UserProfile=0&LanguageId=2'
driver.get(url)      
for k in (74, 73, 86, 93, 107, 108, 109):
    worker = []
    select = Select(driver.find_element('id','Tafsir'))
    selected_option = select.first_selected_option
    print(selected_option.text)
    worker.append(selected_option.text)
    
    for i in range(1, 115):
        url = 'https://www.altafsir.com/Tafasir.asp?tMadhNo=0&tTafsirNo=' + str(k) + '&tSoraNo='+str(i)+'&tAyahNo=1&tDisplay=yes&UserProfile=0&LanguageId=2'
        driver.get(url)
        select = Select(driver.find_element('id','Ayat'))
        ayats = len(select.options)
        print("Ayats", ayats)
        for j in range(1, ayats + 1):
            url = 'https://www.altafsir.com/Tafasir.asp?tMadhNo=0&tTafsirNo=' + str(k) + '&tSoraNo='+str(i)+'&tAyahNo='+str(j)+'&tDisplay=yes&UserProfile=0&LanguageId=2'
            driver.get(url)
            try:
                worker.append(driver.find_element(By.CLASS_NAME, "TextAyah").text)
                worker.append(driver.find_element(By.CLASS_NAME, 'TextResultEnglish').text)
            except:
                #worker.append()
                print("Error in", k, i, j)
        print(i)
    TAFSEERS.append(worker)

Tafsir al-Jalalayn
Ayats 7
1
Ayats 286
2
Ayats 200
3
Ayats 176
4
Ayats 120
5
Ayats 165


# Took 4 hours to run

In [ ]:
len(TAFSEER)

In [ ]:
TAFSEER[0]

In [ ]:
df['Tafseer'] = TAFSEER
df.head()

In [ ]:
df.to_csv("Quran_English_with_Tafseer.csv", index=False)